In [1]:
#!wget https://s3-us-west-2.amazonaws.com/static.pyimagesearch.com/opencv-social-distancing-detector/social-distance-detector.zip

In [2]:
#!unzip social-distance-detector.zip

In [3]:
import numpy as np
import cv2
from scipy.spatial import distance as dist
import argparse
import imutils
import os

In [4]:
LABELS = open("yolo-coco/coco.names").read().strip().split("\n")

In [5]:
net = cv2.dnn.readNetFromDarknet("yolo-coco/yolov3.cfg", "yolo-coco/yolov3.weights")

In [6]:
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [7]:
vs = cv2.VideoCapture("vid1.mp4") # 0 for realtime feed
writer = None

In [8]:
def detect_people(frame, net, ln, personIdx=0):
	(H, W) = frame.shape[:2]
	results = []

	blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
	net.setInput(blob)
	layerOutputs = net.forward(ln)

	boxes = []
	centroids = []
	confidences = []

	for output in layerOutputs:
		for detection in output:
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]

			if classID == personIdx and confidence > 0.3:
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")
				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))
				boxes.append([x, y, int(width), int(height)])
				centroids.append((centerX, centerY))
				confidences.append(float(confidence))

	idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.3, 0.3)
	if len(idxs) > 0:
		for i in idxs.flatten():
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])

			r = (confidences[i], (x, y, x + w, y + h), centroids[i])
			results.append(r)
	return results

In [9]:
count = 0

while True:
  (grabbed, frame) = vs.read()
  if not grabbed:
    break
  
  frame = imutils.resize(frame, width=700)
  results = detect_people(frame, net, ln, personIdx=LABELS.index("person"))
  violate = set()
  
  if len(results) >= 2:
    centroids = np.array([r[2] for r in results])
    D = dist.cdist(centroids, centroids, metric="euclidean")
    for i in range(0, D.shape[0]):
      for j in range(i + 1, D.shape[1]):
        if D[i, j] < 50:
          violate.add(i)
          violate.add(j)
          
  for (i, (prob, bbox, centroid)) in enumerate(results):
    (startX, startY, endX, endY) = bbox
    (cX, cY) = centroid
    color = (0, 255, 0)
    if i in violate:
      color = (0, 0, 255)
    cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
    cv2.circle(frame, (cX, cY), 5, color, 1)
    
  text = "Social Distancing Violations: {}".format(len(violate))
  cv2.putText(frame, text, (10, frame.shape[0] - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)
  
  if writer is None:
    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    writer = cv2.VideoWriter("vid1o.avi", fourcc, 25, (frame.shape[1], frame.shape[0]), True)
  
  if writer is not None:
    writer.write(frame)

  count += 1
  print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
